### Subset Distributions

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from wakeful import log_munger, metrics, virus_total, pipelining, preprocessing
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from bat.dataframe_to_matrix import DataFrameToMatrix
%matplotlib inline

#### dnscat2

In [2]:
df_train_conn_dnscat2 = log_munger.hdf5_to_df('dnscat2-2017-12-31-conn-train', './data')
df_test_conn_dnscat2 = log_munger.hdf5_to_df('dnscat2-2017-12-31-conn-test', './data')
print('train value counts:\n', df_train_conn_dnscat2.label.value_counts())
print('test value counts:\n', df_test_conn_dnscat2.label.value_counts())

FileNotFoundError: File ./data/dnscat2-2017-12-31-conn-train.h5 does not exist

In [3]:
df_train_conn_dnscat2 = df_train_conn_dnscat2.dropna(axis=0, how='any')

NameError: name 'df_train_conn_dnscat2' is not defined

In [ ]:
y_df_train_conn_dnscat2 = df_train_conn_dnscat2.pop('label')
to_matrix = DataFrameToMatrix()
X_matrix_train_conn_dnscat2 = to_matrix.fit_transform(df_train_conn_dnscat2)


### PCA Feature Reduction

In [ ]:
pca = PCA(n_components=6)
pca.fit(X_matrix_train_conn_dnscat2)
print(sum(pca.explained_variance_ratio_), '=', pca.explained_variance_ratio_)
print(pca.n_components_)
X_matrix_train_conn_dnscat2 = pca.fit_transform(X_matrix_train_conn_dnscat2)

In [ ]:
X_matrix_train_conn_dnscat2.shape

### SequentialFeature Reduction

In [ ]:
sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)

In [ ]:
sns.pairplot(df_train_conn_dnscat2, hue='label')

### Distributions

In [ ]:
%ls ./data

In [ ]:
keys = [
        ('iodine-forwarded-2017-12-31-conn-test', 'iodine-forwarded-2017-12-31-conn-train'),
        ('iodine-raw-2017-12-31-conn-test', 'iodine-raw-2017-12-31-conn-train'),
        ('dnscat2-2017-12-31-conn-test', 'dnscat2-2017-12-31-conn-train'),
        ('iodine-forwarded-2017-12-31-dns-test', 'iodine-forwarded-2017-12-31-dns-train'),
        ('iodine-raw-2017-12-31-dns-test', 'iodine-raw-2017-12-31-dns-train'),
        ('dnscat2-2017-12-31-dns-test', 'dnscat2-2017-12-31-dns-train'),]

In [ ]:
train_key = 'iodine-forwarded-2017-12-31-conn-train'
test_key = 'iodine-forwarded-2017-12-31-conn-test'
data_dir='./data'

In [ ]:
train_df = log_munger.hdf5_to_df(key, data_dir)
test_df = log_munger.hdf5_to_df(key, data_dir)

In [ ]:
df.columns

In [ ]:
pipelining.feature_selection_pipeline(train_df=train_df, test_df=test_df)

In [ ]:
df = df[['local_orig', 'local_resp', 'orig_ip_bytes', 'pcr', 'label']]

In [ ]:
df = df.dropna(axis=0, how='any')

In [ ]:
sns.pairplot(df, hue='label')